In [52]:
import os  
import random
import cv2
import numpy as np
from matplotlib import pyplot as plt
from segmentation_models.backbones import get_preprocessing
from keras import backend as K

from image_utils import ImageUtils
from process_results import ProcessResults
from model_utils import ModelUtils

## Initialize

In [53]:
val_frame_path = '/mnt/data/aja/diplomka/disk/val_imgs/val'
val_mask_path = '/mnt/data/aja/diplomka/disk/val_masks/val'

img_height = 512
img_width = 512
backbone = 'resnet18'

preprocess_input = get_preprocessing(backbone)
image_ut = ImageUtils(val_frame_path, val_mask_path, img_height, img_width, architecture='unet')
process_res = ProcessResults()

## Load model

In [64]:
model_RESNET18_150 = ModelUtils.load_model('models/model_unet_disk_pretrainedRESNET18_6')

Loaded model from disk


## Parse and create images for video

In [77]:
path_video = '/mnt/data/aja/diplomka/polcar/Study_02_00103_03_L.avi'
cap = cv2.VideoCapture(path_video)
success, image = cap.read()
totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

In [78]:
idx = 5000

while success:
    success, image = cap.read()
    
    try:
        test_img = preprocess_input(image_ut.preprocess_image(image))/255.
        expand_test = np.expand_dims(test_img, axis=0)
        predicted_img = unet_resnet18_100.predict(expand_test)
        
        filtered_image = process_res.med_ext_med_filt(predicted_img)
        tresholded_image = process_res.tresholding(filtered_image)
        
        contours_image = process_res.find_contours(test_img, tresholded_image)

        idx = idx+1
        print(idx)
        
        cv2.imwrite("/mnt/data/aja/diplomka/polcar/disks_imgs/frame"+str(idx)+".jpg", img*255)
        
    except:
        cap.release()
        cv2.destroyAllWindows()
        print("Complete")
        break

5001
5002
5003
5004
5005
5006
5007
5008
5009
5010
5011
5012
5013
5014
5015
5016
5017
5018
5019
5020
5021
5022
5023
5024
5025
5026
5027
5028
5029
5030
5031
5032
5033
5034
5035
5036
5037
5038
5039
5040
5041
5042
5043
5044
5045
5046
5047
5048
5049
5050
5051
5052
5053
5054
5055
5056
5057
5058
5059
5060
5061
5062
5063
5064
5065
5066
5067
5068
5069
5070
5071
5072
5073
5074
5075
5076
5077
5078
5079
5080
5081
5082
5083
5084
5085
5086
5087
5088
5089
5090
5091
5092
5093
5094
5095
5096
5097
5098
5099
5100
5101
5102
5103
5104
5105
5106
5107
5108
5109
5110
5111
5112
5113
5114
5115
5116
5117
5118
5119
5120
5121
5122
5123
5124
5125
5126
5127
5128
5129
5130
5131
5132
5133
5134
5135
5136
5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200


In [ ]:
!ffmpeg -pattern_type glob -i '*.jpg' -r 5 video2.mp4 